In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

### about shape

In [2]:
x = torch.tensor(1)
y = torch.tensor([1])
z = torch.tensor([[1]])
print(x.shape)
print(y.shape)
print(z.shape)

torch.Size([])
torch.Size([1])
torch.Size([1, 1])


### torch.cat(tensors, dim=0)
Concatenated tensors must have the same shape. <br>
Tensors whose sizes are [] cannot be concatenated.

In [3]:
x = torch.tensor(1)
y = torch.tensor([1])
z = torch.tensor([[1]])
# print(torch.cat([x, x])) Error
print(torch.cat([y, y]))
print(torch.cat([z, z]))

tensor([1, 1])
tensor([[1],
        [1]])


### squeeze and unsqueeze

You should distinguish between tensors of size [x] and tensors of size [1, x]

In [4]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.fc1 = nn.Linear(10, 64)
        self.fc2 = nn.Linear(64, 4)
    
    def forward(self, x):
        h = F.relu(self.fc1(x))
        out = self.fc2(h)
        return out

model = Network()
x = torch.tensor([1,3,2,4,5,1,2,10,2,3]).float()
y = x.unsqueeze(dim=0)
print(f"x's shape : {x.shape}")
print(f"y's shape : {y.shape}")
x_out = model(x)
y_out = model(y)
print(f"x_out : {x_out}")
print(f"y_out : {y_out}")
print(f"x_out's shape : {x_out.shape}")
print(f"y_out's shape : {y_out.shape}")

x's shape : torch.Size([10])
y's shape : torch.Size([1, 10])
x_out : tensor([ 2.8576, -1.1859, -0.6926, -0.4234], grad_fn=<AddBackward0>)
y_out : tensor([[ 2.8576, -1.1859, -0.6926, -0.4234]], grad_fn=<AddmmBackward0>)
x_out's shape : torch.Size([4])
y_out's shape : torch.Size([1, 4])


Especially you should be careful when comparing model's outputs and their target values.

In [5]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.fc1 = nn.Linear(10, 64)
        self.fc2 = nn.Linear(64, 1)
    
    def forward(self, x):
        h = F.relu(self.fc1(x))
        out = self.fc2(h)
        return out

model = Network()
input_data = torch.randint(low=0, high=100, size=(15, 10)).float()
outputs = model(input_data)
print(outputs)

tensor([[  0.9639],
        [ -9.2171],
        [ -9.1887],
        [  3.0913],
        [-10.7156],
        [  4.7492],
        [ -1.5500],
        [ -7.6851],
        [ -6.7137],
        [ -5.3190],
        [-10.1170],
        [ -3.2073],
        [-12.6269],
        [ -7.7646],
        [  5.4059]], grad_fn=<AddmmBackward0>)


In [6]:
target_values = torch.randint(low=-30, high=30, size=(15,))
print(outputs.shape)
print(target_values.shape)

torch.Size([15, 1])
torch.Size([15])


In [7]:
incorrect_mse_loss = (outputs - target_values) ** 2.0
print(incorrect_mse_loss.shape)

torch.Size([15, 15])


In [8]:
outputs = outputs.squeeze()
print(outputs.shape == target_values.shape)
correct_mse_loss = (outputs - target_values) ** 2.0
print(correct_mse_loss.shape)

True
torch.Size([15])


 ### torch.max

one dimentinal array case

In [9]:
one_dimentinal = torch.randn(size=(4,))
one_dimentinal

tensor([-0.5484, -0.8904,  2.0223,  0.9601])

In [10]:
print(torch.max(one_dimentinal))
print(torch.argmax(one_dimentinal))

tensor(2.0223)
tensor(2)


multi dimentional array case

In [11]:
multi_dimentinal = torch.randn(size=(4, 4))
multi_dimentinal

tensor([[ 0.9755,  0.3382,  0.4434, -0.8260],
        [-0.3339, -0.2542,  1.7370, -0.3511],
        [ 0.6496,  1.3269,  1.4022,  1.4153],
        [-1.9361, -0.6137, -0.0198,  1.3351]])

In [12]:
print(torch.max(multi_dimentinal))
print(torch.argmax(multi_dimentinal))

tensor(1.7370)
tensor(6)


In [13]:
print(torch.max(multi_dimentinal, dim=1))

torch.return_types.max(
values=tensor([0.9755, 1.7370, 1.4153, 1.3351]),
indices=tensor([0, 2, 3, 3]))


### torch.gather
torch.gather is convenient when you extract specific action values from the outputs of Q-Network.

In [14]:
class QNetwork(nn.Module):
    def __init__(self):
        super(QNetwork, self).__init__()
        self.fc1 = nn.Linear(128, 64)
        self.fc2 = nn.Linear(64, 4)
    
    def forward(self, x):
        h = F.relu(self.fc1(x))
        out = self.fc2(h)
        return out

model = QNetwork()
x = torch.randn(size=(10, 128))
out = model(x)
print(out)

tensor([[-0.0962,  0.2107, -0.0853, -0.1119],
        [-0.0509, -0.1319, -0.0826, -0.1039],
        [-0.2024, -0.2447, -0.4339, -0.2535],
        [-0.1886, -0.3505, -0.0699, -0.4098],
        [-0.2471, -0.1916, -0.1524, -0.4220],
        [-0.1830, -0.0300,  0.1245, -0.2025],
        [-0.5126, -0.0266, -0.1706, -0.0860],
        [ 0.1305, -0.3697, -0.2185, -0.5217],
        [-0.0516, -0.2895, -0.0609, -0.3497],
        [-0.0044, -0.2623, -0.3063, -0.6574]], grad_fn=<AddmmBackward0>)


In [15]:
action_indexs = torch.tensor([0,1,2,0,3,2,3,2,1,0]).unsqueeze(1)
out.gather(1, action_indexs)

tensor([[-0.0962],
        [-0.1319],
        [-0.4339],
        [-0.1886],
        [-0.4220],
        [ 0.1245],
        [-0.0860],
        [-0.2185],
        [-0.2895],
        [-0.0044]], grad_fn=<GatherBackward0>)

### scatter_
scatter_ can be used when you create two-hot representations of tensors as R2D2 or MuZero.

In [16]:
x = torch.tensor([5.6, 2.4, 3.7, 4.1, 1.5])

In [17]:
ceil = x.ceil().long().unsqueeze(1)
floor = x.floor().long().unsqueeze(1)
ceil_values = (x.ceil() - x).unsqueeze(1)
floor_values = (x - x.floor()).unsqueeze(1)

In [18]:
y = torch.zeros(size=(5, 8))
y.scatter_(dim=1, index=floor, src=floor_values)
y.scatter_(dim=1, index=ceil, src=ceil_values)

# x = torch.tensor([5.6, 2.4, 3.7, 4.1, 1.5])
print(y)

tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.6000, 0.4000, 0.0000],
        [0.0000, 0.0000, 0.4000, 0.6000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.7000, 0.3000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.1000, 0.9000, 0.0000, 0.0000],
        [0.0000, 0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]])


### torch.index_select

In [19]:
x = torch.randn(3, 4)
print("x = ")
print(x)
indices = torch.tensor([0, 2])
print("indices = [0, 2]")
print("dimention 0:")
print(torch.index_select(x, 0, indices))
print("dimention 1:")
print(torch.index_select(x, 1, indices))

x = 
tensor([[ 1.5279,  0.4968,  0.8722, -0.1914],
        [ 0.6810,  0.9399,  0.1952,  0.8909],
        [-0.7884,  1.4546, -0.5774,  0.9142]])
indices = [0, 2]
dimention 0:
tensor([[ 1.5279,  0.4968,  0.8722, -0.1914],
        [-0.7884,  1.4546, -0.5774,  0.9142]])
dimention 1:
tensor([[ 1.5279,  0.8722],
        [ 0.6810,  0.1952],
        [-0.7884, -0.5774]])


### Softmax and Categorical

In [2]:
class Policy(nn.Module):
    def __init__(self):
        super(Policy, self).__init__()
        self.fc1 = nn.Linear(5, 32)
        self.fc2 = nn.Linear(32, 4)
    
    def forward(self, x):
        h = self.fc1(x)
        out = self.fc2(h)
        return out

In [3]:
x = torch.randn(size=(10, 5))
x

tensor([[ 2.0065,  0.3585, -0.1401, -1.0609,  1.2387],
        [ 0.2289, -0.1797, -0.0107,  0.7025,  0.1755],
        [ 0.3215, -0.8379, -0.2751,  0.1788, -0.6690],
        [ 0.4529,  0.9557, -1.0169, -1.6265,  1.1501],
        [-1.1303, -0.0900, -0.0929, -1.5937,  0.2469],
        [-2.2015, -0.2509, -0.5649,  1.9589, -0.7291],
        [ 0.9840, -1.4739, -0.5686, -0.4572, -0.2121],
        [ 0.0443,  0.8285, -0.4780,  0.0955,  0.3003],
        [ 0.1394,  1.3959, -0.0537, -2.4943, -0.7271],
        [ 1.6652,  0.6710, -1.0066,  0.1374,  1.2141]])

In [4]:
model = Policy()
logits = model(x)
print(logits)

tensor([[-0.2479,  1.0703,  0.2728,  0.2918],
        [-0.0620,  0.1799, -0.4320,  0.2425],
        [ 0.0045,  0.3784, -0.3075,  0.2465],
        [-0.2078,  0.6510,  0.0863,  0.1073],
        [-0.4231, -0.0885, -0.2252,  0.3948],
        [ 0.1385, -0.8159, -1.1621,  0.0543],
        [-0.2250,  0.7467, -0.0654,  0.3063],
        [ 0.0812,  0.2425, -0.3339,  0.0854],
        [ 0.1523,  0.5864,  0.2455,  0.2238],
        [ 0.0144,  0.9277, -0.0450,  0.0056]], grad_fn=<AddmmBackward0>)


In [5]:
softmax = F.softmax(logits, dim=1)
print(softmax)

tensor([[0.1229, 0.4593, 0.2069, 0.2109],
        [0.2315, 0.2948, 0.1599, 0.3138],
        [0.2242, 0.3259, 0.1642, 0.2857],
        [0.1647, 0.3887, 0.2210, 0.2257],
        [0.1700, 0.2376, 0.2072, 0.3852],
        [0.3881, 0.1494, 0.1057, 0.3568],
        [0.1535, 0.4055, 0.1800, 0.2610],
        [0.2605, 0.3060, 0.1720, 0.2615],
        [0.2121, 0.3273, 0.2328, 0.2278],
        [0.1843, 0.4594, 0.1737, 0.1827]], grad_fn=<SoftmaxBackward0>)


In [7]:
m = torch.distributions.categorical.Categorical(logits=logits)
m.sample()

tensor([1, 1, 3, 2, 1, 3, 1, 3, 1, 3])